In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import copy
import random
from skimage.feature import local_binary_pattern


In [2]:
def crop_resize_center_dataset(origin_path, crop_path, max_number=5000, crop_shape = (256, 256)):
    floders = os.listdir(origin_path)
    for folder in floders:
        img_path2 = origin_path + folder
        crop_path2 = crop_path + folder
        check_folder(crop_path2)
        
        cur_list = []
        # 影像存檔檔名
        count = 1
        for filename in os.listdir(img_path2):
            if 'jp' in filename:# or 'jpg' in filename):
                # 讀影像
                #print(img_path2 + '/' + filename)
                img = cv2.imread((img_path2 + '/' + filename), cv2.IMREAD_GRAYSCALE)
                cur_shape = img.shape
                x = int(cur_shape[0] / 2 - crop_shape[0] / 2)
                y = int(cur_shape[1] / 2 - crop_shape[1] / 2)
                # 擷取原始影像中800 x 640的部分
                crop_img = img[y:y+crop_shape[0], x:x+crop_shape[1]]
                cv2.imwrite((crop_path2 + '/' + filename), crop_img)
                count += 1
            if count > max_number:
                break
        
        
        np.save(f'{crop_path}/{folder}.npy', cur_list)
        #np.save(f'{crop_path}/{folder}_lbp_noise.npy', cur_noise_list)

In [3]:
def check_folder(path):
    if not os.path.isdir(path):
        os.makedirs(path)

In [ ]:
origin_path = 'C:/Users/kaka/PycharmProjects/pythonProject/AOI/zhiyong_fabric/origin/'
crop_path = 'C:/Users/kaka/PycharmProjects/pythonProject/AOI/ICPR2023/Dataset/ORINGIN/'
crop_resize_center_dataset(origin_path, crop_path)

In [ ]:
def generatorDefect(gray_image, gen_type = 1):
    
    detect_w, detect_h = img1_gray.shape
    
    defect_data = copy.deepcopy(gray_image)
    defect_label = np.zeros(np.shape(defect_data), dtype=np.uint8)  # 返回与图像 img1 尺寸相同的全零数组
    defect_value = random.randrange(0, 255)
    
    if gen_type == 0:
        # ellipse
        # detection's width height
        detect_w = random.randrange(10, 100)
        detect_h = random.randrange(int(detect_w/4), int(detect_w*2))
        # position center's x,y
        detect_x, detect_y = random.randrange(0, detect_w), random.randrange(0, detect_h)
        #水平開始向逆時針偏移
        cur_angle = random.randrange(0, 45)
        #繪製角度由0~180度的
        strat_ellipse = random.randrange(0, 100)
        end_ellipse = random.choice([110, 270,360])

        # 中心座標是(256, 256)，長軸短軸分別是(70, 30)，由水平開始向逆時針偏移15度，繪製角度由0~180度的實心橢圓
        cv2.ellipse(defect_data, (detect_x,detect_y), (detect_w,detect_h), cur_angle, strat_ellipse, end_ellipse, (defect_value), -1)
        cv2.ellipse(defect_label, (detect_x,detect_y), (detect_w,detect_h), cur_angle, strat_ellipse, end_ellipse, (1), -1)
        defect_label
        
    elif gen_type == 1:
        # 畫粗度為7的紅色線
        start_positon = (random.randrange(0,  detect_w),  random.randrange(0,  detect_h))
        end_positon = (random.randrange(0,  detect_w),  random.randrange(0,  detect_h))
        defect_width =  random.randrange(5,  int(w/5))
        cv2.line(defect_data, start_positon,end_positon, (defect_value), defect_width)
        cv2.line(defect_label, start_positon,end_positon, (1), defect_width)

    elif gen_type == 2:
        # detection's width height
        detect_w = random.randrange(10, 100)
        detect_h = random.randrange(int(detect_w/4), int(detect_w*2))
        # position center's x,y
        detect_x, detect_y = random.randrange(0, detect_w), random.randrange(0, detect_h)
        defect_radio = random.randrange(10, 40)
        cv2.circle(defect_data, (detect_x, detect_y), defect_radio, (defect_value), -1)  # -1 表示实心
        cv2.circle(defect_label, (detect_x, detect_y), defect_radio, (1), -1)  # -1 表示实心
    
    #imgAddMask1 = cv2.add(defect_data, np.zeros(np.shape(defect_data), dtype=np.uint8), mask=defect_data)  # 提取圆形 ROI
    
    return defect_data, defect_label

In [ ]:
def matchData(normal_dataset, defect_dataset):
    len_normal = len(normal_data)
    len_defect = len(defect_data)
    
    half_normal_len = int(len_normal/2)
    
    # len, width, height, channel
    normal_data  = np.concatenate((normal_dataset[0:half_normal_len, :,:,: ], normal_dataset[half_normal_len:, :,:,: ]), axis = 3)
    
    normal_label = np.ones(half_normal_len)
    
    defect_data  = np.concatenate((normal_dataset[0:len_defect, :,:,: ], defect_dataset[0:len_defect, :,:,: ]), axis = 3)
    
    defect_label = np.zeros(len_defect)
    
    return normal_data, normal_label, defect_data, defect_label

In [ ]:
def readData(image_path, image_size = (256, 256)):
    all_x_data = []
    
    file_list = os.listdir(image_path)
    for file in file_list:
        img = cv2.imread(image_path + '/' + file, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, image_size, interpolation=cv2.INTER_NEAREST)
        all_x_data.append(img)
    
    all_x_data = np.array(all_x_data).reshape((-1,)+image_size).astype("float32") 
    
    return all_x_data

In [ ]:
def traditionalLbp(img_gray):
    img_lbp= local_binary_pattern(img_gray, 8, 1.0, 'ror')
    return img_lbp

# AITEX dataset


In [13]:
path_normal_dataset = 'C:/Users/kaka/PycharmProjects/pythonProject/dataset/Fabric/AITEX/NODefect_images'
path_defect_dataset = 'C:/Users/kaka/PycharmProjects/pythonProject/dataset/Fabric/AITEX/Defect_images/'
path_mask_dataset = 'C:/Users/kaka/PycharmProjects/pythonProject/dataset/Fabric/AITEX/Mask_images/'
path_save = 'C:/Users/kaka/PycharmProjects/pythonProject/AOI/ICPR2023/Dataset/AITEX/'

def splitTrainImage(folder_path,save_path):
    all_x_data = []
    folder_list = os.listdir(folder_path)
    for floder in folder_list:
        images_list = os.listdir(folder_path + '/' + floder)
        for images in images_list:
            cur_imgae = cv2.imread(folder_path + '/' + floder + '/' + images, 3)
            
            cur_type = images.split('.')[0].split('_')[-1]
            cur_index = images.split('.')[0].split('_')[0]

            save_image_path = f'{save_path}/{cur_type}/TRAIN/defect-free/'
            
            check_folder(save_image_path)
            
            for cur_pos in range(5, 15):
                crop_img = cur_imgae[:, cur_pos * 256 :(cur_pos + 1) * 256]
                cv2.imwrite(f'{save_image_path}/{cur_index}_{cur_pos}.png', crop_img)
                
def splitTestImage(folder_path, mask_path, save_path):
    all_x_data = []
    images_list = os.listdir(folder_path)
    for images in images_list:
        
       
        
        mask_pos = f'{mask_path}/{images.split(".")[0]}_mask.png'
        
#         print(folder_path  + '/' + images)
#         print(mask_pos)
        
        cur_imgae = cv2.imread(folder_path  + '/' + images, 3)
        cur_mask_image = cv2.imread(mask_pos, cv2.IMREAD_GRAYSCALE)
        
        
        cur_mask = np.where(cur_mask_image > 0, 1, 0)
        

#         print(np.unique(cur_mask))
        cur_type = images.split('.')[0].split('_')[-1]
        cur_index = images.split('.')[0].split('_')[0]

        save_image_path = f'{save_path}/{cur_type}/TEST/'

        check_folder(save_image_path)
        check_folder(f'{save_image_path}/defect')
        check_folder(f'{save_image_path}/groundtruth')

        for cur_pos in range(5, 15):
            crop_img = cur_imgae[:, cur_pos * 256 :(cur_pos + 1) * 256]
            crop_msk = cur_mask[:, cur_pos * 256 :(cur_pos + 1) * 256]
            cv2.imwrite(f'{save_image_path}/defect/{cur_index}_{cur_pos}.png', crop_img)
            cv2.imwrite(f'{save_image_path}/groundtruth/{cur_index}_{cur_pos}.png', crop_msk)

splitTrainImage(path_normal_dataset, path_save)

splitTestImage(path_defect_dataset, path_mask_dataset, path_save)

# Fabric dataset

C:\Users\kaka\PycharmProjects\pythonProject\dataset\Fabric\FabricDefect\FabricDataset



In [14]:
path_image = 'C:/Users/kaka/PycharmProjects/pythonProject/dataset/Fabric/FabricDefect/FabricDataset/Images/'
path_mask = 'C:/Users/kaka/PycharmProjects/pythonProject/dataset/Fabric/FabricDefect/FabricDataset/Masks/'
path_save = 'C:/Users/kaka/PycharmProjects/pythonProject/AOI/ICPR2023/Dataset/FabricDataset/'

def splitImage(folder_path, mask_path, save_image_path):
    all_x_data = []
    images_list = os.listdir(folder_path)
    
    
    check_folder(save_image_path)


    check_folder(f'{save_image_path}/defect-free/')
    check_folder(f'{save_image_path}/defect/')
    check_folder(f'{save_image_path}/groundtruth/')
        
    for images in images_list:
        
        
        images_name = images.split('.')[0]
        cur_imgae = cv2.imread(folder_path  + '/' + images)
        
#         print(mask_path +  '/' + images)
        cur_mask_image = cv2.imread(mask_path +  '/' + images_name +'.png',3)        
        cur_mask = np.where(cur_mask_image > 0, 255, 0)


        cur_size = 256
        for i in range(2):
            for j in range(2):
                crop_img = cur_imgae[i * cur_size: (i+1) * cur_size,j * cur_size: (j+1) * cur_size]
                crop_msk = cur_mask[i * cur_size: (i+1) * cur_size,j * cur_size: (j+1) * cur_size]
                
                if crop_msk.sum() > 0:
                    cv2.imwrite(f'{save_image_path}/defect/{images_name}_{i}{j}.png', crop_img)
                    cv2.imwrite(f'{save_image_path}/groundtruth/{images_name}_{i}{j}.png', crop_msk)
                else:
                    cv2.imwrite(f'{save_image_path}/defect-free/{images_name}_{i}{j}.png', crop_img)

splitImage(path_image, path_mask, path_save)

# DAGM2007

Oringin size: 512 * 512
==> 256 * 256

In [15]:
path_base = 'C:/Users/kaka/PycharmProjects/pythonProject/dataset/Fabric/DAGM2007/'

path_save = 'C:/Users/kaka/PycharmProjects/pythonProject/AOI/ICPR2023/Dataset/DAGM2007/'

def splitImage(folder_path, save_image_path, resize_shape = (256, 256, 3)):
    all_x_data = []
    images_list = os.listdir(folder_path)
    
    check_folder(save_image_path)

    
    
    for cur_type in range(1, 11):    
        folder_type = f'Class{cur_type}'
        
        train_normal_path = f'{save_image_path}/{folder_type}/Train/defect-free/'
        train_maskl_path = f'{save_image_path}/{folder_type}/Train/defect-free/'
        test_data_path = f'{save_image_path}/{folder_type}/TEST/defect/'
        test_mask_path = f'{save_image_path}/{folder_type}/TEST/groundtruth'
        
        check_folder(train_normal_path)
        check_folder(test_data_path)
        check_folder(test_mask_path)
    
        
        cur_folder_type_path = f'{folder_path}{folder_type}'
        images_list = os.listdir(f'{cur_folder_type_path}/Train/IMAGE')
        for images_full_name in images_list:
            images_name, file_suffix = images_full_name.split('.')[0], images_full_name.split('.')[1]
            
            print(f'{cur_folder_type_path}/Train/IMAGE/' + images_full_name)
            cur_data_image = cv2.imread(f'{cur_folder_type_path}/Train/IMAGE/{images_full_name}', 3) 

            cur_maks_pos = f'{cur_folder_type_path}/Train/Label/{images_name}_label.PNG'
#             print(cur_maks_pos)
            if os.path.exists(cur_maks_pos): 
                cv2.imwrite(f'{train_normal_path}{images_full_name}', cur_data_image)
            else:
                cv2.imwrite(f'{train_normal_path}{images_full_name}', cur_data_image)
                cv2.imwrite(f'{train_maskl_path}{images_full_name}', cur_maks_pos)
                
           

        print((f'{cur_folder_type_path}/Test/IMAGE/'))
        images_list = os.listdir(f'{cur_folder_type_path}/Test/IMAGE/')        
        for images_name in images_list:
            cur_data_image = cv2.imread(f'{test_data_path}{images_name}', 3) 
            
            cv2.imwrite(f'{test_mask_path}{images_name}', cur_data_image)

splitImage(path_base, path_save)
'C:/Users/kaka/PycharmProjects/pythonProject/dataset/Fabric/DAGM2007/Class1/Train/IMAGE/0576.PNG'

C:/Users/kaka/PycharmProjects/pythonProject/dataset/Fabric/DAGM2007/Class1/Train/IMAGE/0576.PNG


error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'imwrite'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'


In [17]:
cur_data_image = cv2.imread('C:/Users/kaka/PycharmProjects/pythonProject/dataset/Fabric/DAGM2007/Class1/Train/IMAGE/0576.PNG', 3)

cv2.imwrite(f'C:/Users/kaka/PycharmProjects/pythonProject/AOI/ICPR2023/Dataset/1.png', cur_data_image)

True

# MVTEX


1024 * 1024 ==>   256 * 256


In [18]:
path_base = 'C:/Users/kaka/PycharmProjects/pythonProject/dataset/Fabric/MVTEC/'

path_save = 'C:/Users/kaka/PycharmProjects/pythonProject/AOI/ICPR2023/Dataset/MVTEC/'

def splitImage(folder_path, save_path):
    images_folder = ['carpet', 'leather', 'wood']
    
    image_shape = 256
        
    for cur_folder in images_folder:
        
        save_train_path = f'{save_path}{cur_folder}/TRAIN/defect-free'
        save_test_defect_path = f'{save_path}/{cur_folder}/TEST/defect/'
        save_test_normal_path = f'{save_path}/{cur_folder}/TEST/defect/-free'
        save_mask_path = f'{save_path}/{cur_folder}/TEST/groundtruth/'
        
        check_folder(save_train_path)
        check_folder(save_test_defect_path)
        check_folder(save_test_normal_path)
        check_folder(save_mask_path)
        
        read_mask_path = f'{path_base}/{cur_folder}/ground_truth/'
        read_test_path = f'{path_base}/{cur_folder}/test/'
        read_train_path = f'{path_base}/{cur_folder}/train/good/'
        
        
        images_good_list = os.listdir(read_train_path)
        
        for images_full_name in images_good_list:
            images_name = images_full_name.split(".")[0]
            cur_imgae = cv2.imread(f'{read_train_path}/{images_full_name}', 3) 
            for i in range(4):
                for j in range(4):
                    crop_img = cur_imgae[i * image_shape: (i+1) * image_shape,j * image_shape: (j+1) * image_shape]                    
                    cv2.imwrite(f'{save_train_path}/{images_name}_{i}{j}.png', crop_img)
                    
        
        test_files_list = os.listdir(read_test_path)
        
        
#         print(read_test_path)
        
        for test_file in test_files_list:
            
            test_images_list = os.listdir(f'{read_test_path}/{test_file}')
            for images_full_name in test_images_list:
                images_name = images_full_name.split(".")[0]
                
                cur_imgae = cv2.imread(f'{read_test_path}/{test_file}/{images_full_name}', 3) 
                
                mask_pos = f'{read_mask_path}/{test_file}/{images_name}_mask.png'
                
#                 print(mask_pos)
                cur_mask = cv2.imread(mask_pos, cv2.IMREAD_GRAYSCALE)
                
                for i in range(4):
                    for j in range(4):
                        crop_img = cur_imgae[i * image_shape: (i+1) * image_shape,j * image_shape: (j+1) * image_shape]
                        
                        if test_file == 'good':
                            cv2.imwrite(f'{save_test_normal_path}/{test_file}_{images_name}_{i}{j}.png', crop_img)
                            crop_msk = np.zeros((256,256), dtype=np.uint8)
                        else:
                            cv2.imwrite(f'{save_test_defect_path}/{test_file}_{images_name}_{i}{j}.png', crop_img)
                            crop_msk = cur_mask[i * image_shape: (i+1) * image_shape,j * image_shape: (j+1) * image_shape]
                            cv2.imwrite(f'{save_mask_path}/{test_file}_{images_name}_{i}{j}.png', crop_msk)


splitImage(path_base, path_save)

# YDFID -1


In [39]:
path_base = 'C:/Users/kaka/PycharmProjects/pythonProject/dataset/Fabric/YDFID-1/'

path_save = 'C:/Users/kaka/PycharmProjects/pythonProject/AOI/ICPR2023/Dataset/YDFID-1/'

def splitImage(folder_path, save_path):
    first_folder = os.listdir(folder_path)
    
#     first_folder = ['SL']
            
    for cur_folder in first_folder:
        
        second_folders = os.listdir(f'{folder_path}/{cur_folder}')
#         first_folder = ['SL16']
        
        for cur_second_folder in second_folders:
        
            save_train_path = f'{save_path}/{cur_second_folder}/TRAIN/defect-free'
            save_normal_path = f'{save_path}/{cur_second_folder}/TEST/defect-free/'
            save_defect_path = f'{save_path}/{cur_second_folder}/TEST/defect/'
            save_mask_path = f'{save_path}/{cur_second_folder}/TEST/groundtruth/'

            check_folder(save_train_path)
            check_folder(save_normal_path)
            check_folder(save_defect_path)
            check_folder(save_mask_path)

            read_mask_path = f'{path_base}/{cur_folder}/{cur_second_folder}/test/ground truth/'
            read_test_defect_path = f'{path_base}/{cur_folder}/{cur_second_folder}/test/defect/'
            read_test_normal_path = f'{path_base}/{cur_folder}/{cur_second_folder}/test/defect-free/'
            read_train_path = f'{path_base}/{cur_folder}/{cur_second_folder}/train/defect-free/'


            images_good_list = os.listdir(read_train_path)

            for images_full_name in images_good_list:
                images_name = images_full_name.split(".")[0]
                cur_image = cv2.imread(f'{read_train_path}/{images_full_name}', 3) 
                cv2.imwrite(f'{save_train_path}/{images_full_name}', cur_image)


            test_defect_files_list = os.listdir(read_test_defect_path)
            for images_full_name in test_defect_files_list:  
                
                images_name = images_full_name.split(".")[0]
                
                
                cur_image = cv2.imread(f'{read_test_defect_path}/{images_full_name}', 3) 
                
                
                
                cur_mask_pos = f'{read_mask_path}/{images_name}.png'
                
                if not os.path.exists(cur_mask_pos): 
                    cur_mask_pos =  f'{read_mask_path}/residual_{images_name}.png'
                
                
#                 print(cur_mask_pos)
                cur_mask = cv2.imread(f'{cur_mask_pos}', cv2.IMREAD_GRAYSCALE)


                cv2.imwrite(f'{save_defect_path}/{images_full_name}', cur_image)
                cv2.imwrite(f'{save_mask_path}/{images_full_name}', cur_mask)

                
            
            test_files_normal_list = os.listdir(read_test_normal_path)
            for images_full_name in test_files_normal_list:                
                cur_image = cv2.imread(f'{read_test_normal_path}/{images_full_name}', 3) 
                cv2.imwrite(f'{save_normal_path}/{images_full_name}', cur_image)

# splitImage(path_base, path_save)